# McDonalds Diet Problem

In this notebook, we will determine the minimum cost McDonalds diet

In [5]:
foods = [:QP, :MD, :BM, :FF, :MC, :FR, :SM, :M1, :OJ]
# Word to the wise---Julia symbols cannot start with a number (M1 instead of 1M. It took me a while to figure this out)

nutrients = [:Prot, :VitA, :VitC, :Calc, :Iron, :Cals, :Carb]

cost = Dict(zip(foods,[1.84,2.19,1.84,1.44,2.29,0.77,1.29,0.6,0.72]))
required = Dict(zip(nutrients,[55,100,100,100,100,2000,350]))
using NamedArrays
A = [
    28 24 25 14 31 3 15 9 1
    15 15 6 2 8 0 4 10 2
    6 10 2 0 15 15 0 4 120
    30 20 25 15 15 0 20 30 2
    20 20 20 10 8 2 15 0 2
    510 370 500 370 400 220 345 110 80
    34 33 42 38 42 26 27 12 20
]
A_NA = NamedArray(A, (nutrients,foods), ("Nutrients","Menu Items")) ;



Some new requirements 

Helen will allow me to eat as many burgers as a want, so I can maximize the number of burgers, but 

* We now have some maximum amount of every nutrient (say twice the minimum requirement)

* I can't let the number of calories I eat get "out of proportion" to my vitamin levels.
Specifically 

(cals eaten)/(vitc eaten) <= $\rho$

(cals eaten)/(vita eaten) <= $\rho$

for $\rho = 40$


In [6]:
burgers = [:QP, :MD, :BM]
# Fancy dict comprehension to create max_allowed
max_allowed = Dict( x => 3*required[x] for x in keys(required))

p = 40 ;

In [7]:
using JuMP, HiGHS
m = Model(HiGHS.Optimizer)
@variable(m, x[foods] >= 0)
@objective(m, Max, sum(x[j] for j in burgers))
@constraint(m, [i in nutrients], sum(A_NA[i,j]*x[j] for j in foods) >= required[i])

# Add max nutrients 
@constraint(m, [i in nutrients], sum(A_NA[i,j]*x[j] for j in foods) <= max_allowed[i])

# Let's use 'expressions'
@expression(m, cals_eaten, sum(A_NA[:Cals,j]*x[j] for j in foods))
@expression(m, vitc_eaten, sum(A_NA[:VitC,j]*x[j] for j in foods))
@expression(m, vita_eaten, sum(A_NA[:VitA,j]*x[j] for j in foods))

# Do the ratio constraints
@constraint(m, vitc_ratio, cals_eaten / vitc_eaten <= p)
@constraint(m, vita_ratio, cals_eaten / vita_eaten <= p)

print(m)

# If you don't want the solver output, you can make it slient like this
set_silent(m)
optimize!(m)

# You can use the Julia Formatting library for prettier printing, specifying precision, or you can use @printf like C
using Formatting
printfmtln("Minimum cost menu \${:.2f}: ", objective_value(m))
for j in foods
    if value(x[j]) > 1.0e-6
        printfmtln("Eat {1:.2f} of menu item {2}", value(x[j]), j)
    end
end



ErrorException: Constraints of type MathOptInterface.ScalarNonlinearFunction-in-MathOptInterface.LessThan{Float64} are not supported by the solver.

If you expected the solver to support your problem, you may have an error in your formulation. Otherwise, consider using a different solver.

The list of available solvers, along with the problem types they support, is available at https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers.

Oh Dear, this was not a linear program, so HiGHS could not solve it.

(cals eaten)/(vitc eaten) <= $\rho$

(cals eaten)/(vita eaten) <= $\rho$

But since vitc eaten and vita eaten are both > 0, we can multiply through by the denominator and write it as a linear program:

(cals eaten) <= (vitc eaten) * $\rho$

(cals eaten) <= (vita eaten) * $\rho$

In [8]:
using JuMP, HiGHS
m = Model(HiGHS.Optimizer)
@variable(m, x[foods] >= 0)
@objective(m, Max, sum(x[j] for j in burgers))
@constraint(m, [i in nutrients], sum(A_NA[i,j]*x[j] for j in foods) >= required[i])

# Add max nutrients 
@constraint(m, [i in nutrients], sum(A_NA[i,j]*x[j] for j in foods) <= max_allowed[i])

# Let's use 'expressions'
@expression(m, cals_eaten, sum(A_NA[:Cals,j]*x[j] for j in foods))
@expression(m, vitc_eaten, sum(A_NA[:VitC,j]*x[j] for j in foods))
@expression(m, vita_eaten, sum(A_NA[:VitA,j]*x[j] for j in foods))

# Do the ratio constraints as LP
@constraint(m, vitc_ratio, cals_eaten <= vitc_eaten * p)
@constraint(m, vita_ratio, cals_eaten <= vita_eaten * p)


# If you don't want the solver output, you can make it slient like this
#set_silent(m)
optimize!(m)

# You can use the Julia Formatting library for prettier printing, specifying precision, or you can use @printf like C
using Formatting
printfmtln("Minimum cost menu \${:.2f}: ", objective_value(m))
for j in foods
    if value(x[j]) > 1.0e-6
        printfmtln("Eat {1:.2f} of menu item {2}", value(x[j]), j)
    end
end



Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
16 rows, 9 cols, 133 nonzeros
9 rows, 9 cols, 75 nonzeros
9 rows, 9 cols, 75 nonzeros
Presolve : Reductions: rows 9(-7); columns 9(-0); elements 75(-58)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.2499927390e-02 Ph1: 9(56.3945); Du: 3(0.0624999) 0s
          5    -5.9856443719e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 5
Objective value     :  5.9856443719e+00
HiGHS run time      :          0.00
Minimum cost menu $5.99: 
Eat 5.99 of menu item :MD
Eat 4.43 of menu item :FR
Eat 0.74 of menu item :M1
Eat 1.42 of menu item :OJ
